In [146]:
##############################
#Chris OWID JDS exercise     #
#Maintainer: Christopher Chan#
#Version: 0.0.1              #
#Date: 2023-02-26            #
##############################

import os, sys, re
import statsmodels
import pathlib

import numpy as np
import pandas as pd
import openpyxl as opx
import scipy as sp
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

data_raw = pathlib.Path('../data/01_raw/')
data_intermediate = pathlib.Path('../data/02_intermediate/')
data_output = pathlib.Path('../data/07_model_output/')

## GOAL: Calculate both Crude Death Rate and Age-Standardised Death Rates from Age-Specific Death Rates
### Data:
1. UN WPP Population Estiamtes (1950-2021) (UN DESA., 2022)
2. WHO Standard Population Distribution aggregate (Ahmad et al., 2001)
3. Age-Specific Death Rates COPD 2019

### Definitions (ONS., 2023):
> Age-Specific Death Rates: Number of deaths in hte age group per 1,000 population in the same age group\
> Crude Death Rate: Total deaths per 1,000 population\
> Age-Standardised Death Rates: 

In [147]:
# Data ingest
# pandas 2.0

# Loop ingestion of various WPP pop estimate variants
WPP = {}
with pd.ExcelFile(f"{data_raw}/WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_REV1.xlsx") as WPP_wb:
    for i in WPP_wb.sheet_names:
        if i != 'NOTES':
            st_name = i.replace(' ', '_')
            WPP[st_name] = pd.read_excel(f"{data_raw}/WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_REV1.xlsx",
                                         sheet_name = i, skiprows = np.arange(0, 16, 1).tolist(), dtype_backend = 'pyarrow')

assert WPP["Estimates"].columns.tolist() == WPP["Instant-replacement_zero_migr"].columns.tolist(), "Columns names do not match!"

# Other datasets
with open(f"{data_raw}/WHO_ASR.csv") as WHO_df:
    WHO_SPD = pd.read_csv(WHO_df, delimiter = ',', engine = 'pyarrow', dtype_backend = 'pyarrow')
with open(f"{data_raw}/age_specific_COPD.csv") as AS_df:
    AS_COPD = pd.read_csv(AS_df, delimiter = ',', engine = 'pyarrow', dtype_backend = 'pyarrow')

In [148]:
WPP["High_variant"][WPP["High_variant"]["Year"] == 2022]

,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,...,"Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)","Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)","Mortality between Age 15 and 50, both sexes (deaths under age 50 per 1,000 alive at age 15)","Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 50 (deaths under age 50 per 1,000 females alive at age 15)","Mortality between Age 15 and 60, both sexes (deaths under age 60 per 1,000 alive at age 15)","Male Mortality between Age 15 and 60 (deaths under age 60 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 60 (deaths under age 60 per 1,000 females alive at age 15)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)"
0,1,High,WORLD,<NA>,900,<NA>,<NA>,1,World,0,...,224.064,155.558,78.931,95.683,61.297,152.967,185.961,118.483,0,0
80,81,High,Sub-Saharan Africa,b,1834,<NA>,<NA>,202,SDG region,1828,...,400.475,328.908,175.682,195.913,155.235,300.363,335.962,264.788,-450.072,-0.385
159,160,High,Northern Africa and Western Asia,<NA>,1833,<NA>,<NA>,747,SDG region,1828,...,177.432,114.587,53.64,66.058,39.699,122.67,150.803,91.157,40.909,0.074
238,239,High,Central and Southern Asia,<NA>,1831,<NA>,<NA>,62,SDG region,1828,...,260.507,191.657,87.143,104.97,68.233,194.173,227.863,158.2,-1270.348,-0.608
317,318,High,Eastern and South-Eastern Asia,<NA>,1832,<NA>,<NA>,753,SDG region,1828,...,144.132,85.666,47.193,58.903,34.676,100.039,127.284,71.487,-322.943,-0.138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22203,22204,High,Samoa,<NA>,882,WSM,WS,882,Country/Area,957,...,191.932,124.525,62.983,76.674,48.685,141.627,172.458,107.452,-1.5,-6.711
22282,22283,High,Tokelau,35,772,TKL,TK,772,Country/Area,957,...,138.518,137.223,61.563,57.267,65.838,129.968,130.622,130.308,0,0
22361,22362,High,Tonga,<NA>,776,TON,TO,776,Country/Area,957,...,240.088,162.637,93.789,111.406,77.66,189.669,227.862,151.131,-0.8,-7.446
22440,22441,High,Tuvalu,<NA>,798,TUV,TV,798,Country/Area,957,...,393.821,230.433,166.314,209.391,115.979,302.188,374.681,209.364,-0.06,-5.28


## Calculate Crude Death Rate for US and Uganda 2019

> CDR = (Total deaths / Total pop) * 1000

Since there are no 2019 data for the UN WPP population estimate, I will use cubic spline to interpolate the 2019 figures

In [149]:
# Single sample test
test_df = WPP["High_variant"][(WPP["High_variant"]["ISO3 Alpha-code"] == "UGA")]
linspace = np.arange(2019, max(test_df["Year"]))

test_df_cs = sp.interpolate.CubicSpline(test_df["Year"], test_df["Total Deaths (thousands)"])
res = test_df_cs(2019)

print(res)

237.2977484702041


In [151]:
def UNWPP_interp_CDR(df_dict, target_year):

    df = pd.DataFrame({"Country": np.tile(["USA", "UGA"], len(df_dict.keys())),
                       "Variant_estimate": np.repeat(list(df_dict.keys()), 2),
                       "CDR": np.nan})

    cdr_values = []

    for k in df_dict.keys():
        print(f"Calculation variant {k}")
        USA = df_dict[k][df_dict[k]["ISO3 Alpha-code"] == "USA"]
        UGA = df_dict[k][df_dict[k]["ISO3 Alpha-code"] == "UGA"]

        # USA cubic spline
        USA_vTD_cs = sp.interpolate.CubicSpline(USA["Year"], USA["Total Deaths (thousands)"])
        USA_vTD = USA_vTD_cs(target_year)
        USA_vTP_cs = sp.interpolate.CubicSpline(USA["Year"], USA["Total Population, as of 1 July (thousands)"])
        USA_vTP = USA_vTP_cs(target_year)
        USA_vCDR = USA_vTD / USA_vTP

        # UGA cubic spline
        UGA_vTD_cs = sp.interpolate.CubicSpline(UGA["Year"], UGA["Total Deaths (thousands)"])
        UGA_vTD = UGA_vTD_cs(target_year)
        UGA_vTP_cs = sp.interpolate.CubicSpline(UGA["Year"], UGA["Total Population, as of 1 July (thousands)"])
        UGA_vTP = UGA_vTP_cs(target_year)
        UGA_vCDR = UGA_vTD / UGA_vTP

        print(f"For year {target_year}, USA {k} Crude Death Rate is {USA_vCDR} and UGA {k} Crude Death Rate is {UGA_vCDR}")

        #vCDR = [USA_vCDR, UGA_vCDR]

        cdr_values.append([USA_vCDR, UGA_vCDR])

    df["CDR"] = np.concatenate(cdr_values)
        
    return df

interpDF_2019 = UNWPP_interp_CDR(WPP, 2019)
print(interpDF_2019)
interpDF_2019.to_csv(f'{data_output}/vCDR_2019.csv', sep = ',')

Calculation variant Estimates
For year 2019, USA Estimates Crude Death Rate is 0.008309855030935348 and UGA Estimates Crude Death Rate is 0.005817516929349825
Calculation variant Medium_variant
For year 2019, USA Medium_variant Crude Death Rate is 0.0266573578708496 and UGA Medium_variant Crude Death Rate is 0.005619135856514406
Calculation variant High_variant
For year 2019, USA High_variant Crude Death Rate is 0.0267620720408334 and UGA High_variant Crude Death Rate is 0.005524911299134444
Calculation variant Low_variant
For year 2019, USA Low_variant Crude Death Rate is 0.026553694977010114 and UGA Low_variant Crude Death Rate is 0.00571209664404863
Calculation variant Constant-fertility
For year 2019, USA Constant-fertility Crude Death Rate is 0.02666023166893956 and UGA Constant-fertility Crude Death Rate is 0.005668146658337973
Calculation variant Instant-replacement
For year 2019, USA Instant-replacement Crude Death Rate is 0.026829396077361727 and UGA Instant-replacement Crude 

In [16]:
AS_COPD

,Age group (years),"Death rate, United States, 2019","Death rate, Uganda, 2019"
0,0-4,0.04,0.4
1,5-9,0.02,0.17
2,10-14,0.02,0.07
3,15-19,0.02,0.23
4,20-24,0.06,0.38
5,25-29,0.11,0.4
6,30-34,0.29,0.75
7,35-39,0.56,1.11
8,40-44,1.42,2.04
9,45-49,4.0,5.51
